## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [7]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


In [8]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)


170500096/170498071 [==============================] - 275s 2us/step
(50000, 32, 32, 3)


In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(3,3))
plt.imshow(x_train[0]) 

In [9]:
## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test, mean, std
    
    
## 標準化train & test data    
x_train, x_test, mean_train, std_train = normalize(x_train, x_test) 

In [11]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot = OneHotEncoder()
y_train = one_hot.fit_transform(y_train).toarray()
y_test = one_hot.transform(y_test).toarray()

c:\users\jaspe\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [14]:
classifier=Sequential()

#卷積組合
#32,3,3,input_shape=(32,32,3),activation='relu'
classifier.add(Convolution2D(32, kernel_size=(3, 3), input_shape=(32, 32, 3), activation="relu", padding="same"))
classifier.add(BatchNormalization())

classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(32, kernel_size=(3, 3), input_shape=(32, 32, 3), activation="relu", padding="same"))
classifier.add(BatchNormalization())

In [15]:
#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=100, activation="relu")) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10, activation="softmax"))

c:\users\jaspe\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100)`
  """
c:\users\jaspe\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  


In [16]:
#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=5)

Epoch 1/5
50000/50000 [==============================] - 194s 4ms/step - loss: 1.2591 - acc: 0.5599
Epoch 2/5
50000/50000 [==============================] - 194s 4ms/step - loss: 0.8091 - acc: 0.7152
Epoch 3/5
50000/50000 [==============================] - 196s 4ms/step - loss: 0.5779 - acc: 0.7970
Epoch 4/5
50000/50000 [==============================] - 199s 4ms/step - loss: 0.3939 - acc: 0.8643
Epoch 5/5
50000/50000 [==============================] - 208s 4ms/step - loss: 0.2528 - acc: 0.9134


## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [17]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[1.2385164e-03, 1.6623120e-05, 4.3648031e-01, 4.1231492e-01,
        1.4812058e-01, 1.5318847e-03, 4.4721819e-05, 9.5148942e-05,
        1.5412115e-04, 3.1094085e-06]], dtype=float32)

In [ ]:
## note

# convolution -> (maxpooling降低尺度強化特徵) -> BN -> activation function -> Flatten or GlobalAveragePooling -> fC
# 隨著越來越常用relu, 所以有人先用AF再BN
# FC盡量用少層一點因為會用到大量參數, 最後一層要用跟分類類別一樣多的神經元
# softmax可轉換為機率值(0~1), 適合用在分類問題

## BN benefit
# 參數影響對他小
# 有regulation的效果
